# Data prepration

In [6]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from functions import mean_gen, varx_gen, vary_gen, spherical_data, discrete_angles

In [7]:
n = 100000


# m_th = 40
# m_ph = 40
# thetas = discrete_angles(n, m_th, 2)
# phis = discrete_angles(n, m_ph, 1)
# name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))

theta = 2
phi = 2
thetas = theta * np.ones(n)
phis = phi * np.ones(n)
name = ('discrete_model_th%.0f_ph%.0f' %(theta, phi))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 0.6966621   0.64602116  2.          2.        ]
 [ 1.21093747  0.81166126  2.          2.        ]
 [ 2.05257671  0.06592049  2.          2.        ]
 ...
 [ 0.34160153  0.66658316  2.          2.        ]
 [ 0.66673211 -3.24127035  2.          2.        ]
 [ 2.29239147 -0.44680839  2.          2.        ]]


# Training Interpolated Model

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=20
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-08 12:04:58.419229: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-08 12:04:58.419265: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-08 12:04:58.419507: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 20/160 [==>...........................] - ETA: 1s - loss: 0.5996 - accuracy: 0.6780

2022-08-08 12:04:58.979348: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-08 12:04:58.979385: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-08 12:04:59.005685: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-08 12:04:59.006705: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-08 12:04:59.018972: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220808-120458/train/plugins/profile/2022_08_08_12_04_59

2022-08-08 12:04:59.024017: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220808-120458/train/plugins/profile/2022_08_08_12_04_59/jupyter-matan.trace.json.gz
2022-08-08 12:04:59.026139: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220808-120

160/160 [==============================] - 1s 6ms/step - loss: 0.5438 - accuracy: 0.7224
Epoch 2/20
160/160 [==============================] - 1s 6ms/step - loss: 0.5330 - accuracy: 0.7305
Epoch 3/20
160/160 [==============================] - 1s 6ms/step - loss: 0.5320 - accuracy: 0.7314
Epoch 4/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5315 - accuracy: 0.7308
Epoch 5/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5312 - accuracy: 0.7310
Epoch 6/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5314 - accuracy: 0.7305
Epoch 7/20
160/160 [==============================] - 1s 4ms/step - loss: 0.5311 - accuracy: 0.7310
Epoch 8/20
160/160 [==============================] - 1s 4ms/step - loss: 0.5313 - accuracy: 0.7317
Epoch 9/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5308 - accuracy: 0.7310
Epoch 10/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5308 - accuracy: 0.7310
Epoch 11/2

In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)